# Fine-tune Llama 2 in Google Colab
This notebook runs on a T4 GPU.

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
filepath = "/content/drive/MyDrive/hackathon/train.dat"
df = pd.read_table(filepath, names = ['idx', 'text'])
test_df = df[10000:]
df = df[:10000]
df.head(5),len(df)

(   idx                                               text
 0    4  Catheterization laboratory events and hospital...
 1    5  Renal abscess in children. Three cases of rena...
 2    2  Hyperplastic polyps seen at sigmoidoscopy are ...
 3    5  Subclavian artery to innominate vein fistula a...
 4    4  Effect of local inhibition of gamma-aminobutyr...,
 10000)

In [5]:
import random
# Equally choose labels from dataset
final_df = pd.DataFrame(columns=df.columns)
for group_value in df['idx'].unique():
    group = df[df['idx'] == group_value]
    sample = group.sample(n=250, replace=True, random_state=42)  # Adjust random_state as needed
    final_df = pd.concat([final_df, sample])
# Reset the index of the final DataFrame
final_df.reset_index(drop=True, inplace=True)

# Now final_df contains 100 rows with 25 samples from each "ix" value
print(len(final_df))
print(final_df.head(1000))

1250
    idx                                               text
0     4  Longitudinal dissociation of atrioventricular ...
1     4  Role of perfusion pressure and flow in major o...
2     4  Hemodynamic effects of a new right ventricular...
3     4  Emergency cardiopulmonary bypass support in pa...
4     4  Axillary subclavian vein thrombosis. Changing ...
..   ..                                                ...
995   1  Adoptive immunotherapy of human pancreatic can...
996   1  Adjuvant cyclophosphamide, methotrexate, and f...
997   1  Systemic preoperative chemotherapy with cispla...
998   1  Growth of group A rotaviruses in a human liver...
999   1  Suprasellar pituitary adenoma arising from the...

[1000 rows x 2 columns]


In [6]:
# Make it into single line texts with special tags
df = final_df.assign(text=final_df['text'] + "<CLSFY>" + final_df['idx'].astype(str))
df = df.drop("idx",axis=1)
df.head(4)

,text
0,Longitudinal dissociation of atrioventricular ...
1,Role of perfusion pressure and flow in major o...
2,Hemodynamic effects of a new right ventricular...
3,Emergency cardiopulmonary bypass support in pa...


In [7]:
## Load into required format
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text'],
    num_rows: 1250
})

In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-7b-medical"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("/content/drive/MyDrive/hackathon/"+new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.784300
50,1.709900
75,1.516000
100,1.619200
125,1.476000
150,1.640400
175,1.471700
200,1.574200
225,1.457400
250,1.595600


In [10]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
def predict_func(testls,labels):
  preds = []
  for i in range(len(testls)):
    inp,label = testls[i],labels[i]
    result = pipe(f"{inp} <CLSFY>")
    preds.append(result)
    labels.append(label)
  return preds,labels

test_strings = test_df["text"].values.tolist()
test_labels = test_df["idx"].values.tolist()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1,return_full_text=False)
preds,labels = predict_func(test_strings[:3],test_labels[:3])

In [19]:
# after reloading
for i in range(3):
  print(preds[i])
  print(labels[i],"\n")

[{'generated_text': '5'}]
5 

[{'generated_text': '5'}]
5 

[{'generated_text': '5'}]
5 



In [ ]:
# Empty VRAM
del model
del pipe
# del trainer
import gc
gc.collect()
gc.collect()

## Testing

In [11]:
# Reload model in FP16 and merge it with LoRA weights
new_model = "llama-2-7b-medical"
# /content/drive/MyDrive/hackathon/llama-2-7b-medical
model_name = "NousResearch/Llama-2-7b-chat-hf"
# Load the entire model on the GPU 0
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/hackathon/"+new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1,return_full_text=False)

In [23]:
def predict_func(testls,labels):
  preds = []
  for i in range(len(testls)):
    inp,label = testls[i],labels[i]
    result = pipe(f"{inp} <CLSFY>")
    preds.append(result)
    labels.append(label)
  return preds,labels

test_strings = test_df["text"].values.tolist()
test_labels = test_df["idx"].values.tolist()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1,return_full_text=False)
preds,labels = predict_func(test_strings[:100],test_labels[:100])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [31]:
acc = 0
for i in range(len(preds)):
  # print(test_labels[i],preds[i][0]['generated_text'])
  if str(test_labels[i])==preds[i][0]['generated_text']:
    acc +=1
print(f"Accuracy: {acc/len(preds):.2f}")

Accuracy: 0.93


In [14]:
print(test_strings[0])
print(pipe(f"{test_strings[0]} <CLSFY>",))

Freezing lesions of the developing rat brain: a model for cerebrocortical microgyria. Cerebrocortical microgyri were induced by placing a freezing probe on the skull of P0 and P1 rat pups. Freezing lesions resulted in laminar necrosis of the infragranular layers and the subsequent migration of supragranular neurons through the region of damage. The result was most often a region of four-layered microgyric cortex consisting of a molecular layer, a thickened layer ii, a lamina dissecans (corresponding to the necrotized layers IV, V, and VIa), and a neuronal layer iv which corresponded to layer VIb of the intact cortex. Immunocytochemical investigation of the microgyric cortex with antibodies to neurofilament, glial fibrillary acidic protein and glutamate showed more widespread disruption of neocortical architecture than could be seen from Nissl preparations. In contrast, vasoactive intestinal peptide-containing neuronal bodies appeared to be distributed normally in the microgyric region 

In [ ]:
# 1: neoplasms
# 2: digestive system diseases
# 3: nervous system diseases
# 4: cardiovascular diseases
# 5: general pathological conditions